In [1]:
import numpy as np
import os
import time

In [2]:
import keras

In [3]:
keras.__version__

'2.4.3'

In [4]:
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [5]:
#!pip install -q -U tensorflow-addons==0.11.2

### Utilities

In [6]:
from utils import verifyDir
from utils.networks import normalize, unnormalize, plot_data

### Dataset

In [7]:
from utils.CIFAR10 import load_real_samples

### Discriminator & Generator

In [8]:
from utils.CIFAR10 import define_discriminator
from utils.CIFAR10 import define_generator

### Semi-Supervised GAN

In [9]:
from utils.networks import define_gan

### Selecting sub-set 

In [10]:
from utils.networks import select_supervised_samples, generate_real_samples
from utils.networks import generate_fake_samples, generate_latent_points

### Training

In [11]:
# train the generator and discriminator
def train(generator_model, unsupervised_model, supervised_model, gan_model, dataset_train, dataset_test, 
          latent_dim=100, n_epochs=20, n_batch=100, percent_samples=1.0, n_classes=10):
    
    # select supervised dataset_train
    X_sup, y_sup = select_supervised_samples(dataset_train, percent_samples=percent_samples, n_classes=n_classes)
    print("Sup samples:", X_sup.shape, y_sup.shape)
    
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset_train[0].shape[0] / n_batch)
    
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    print('n_epochs=%d, n_batch=%d, batch/epoch=%d, steps=%d' % (n_epochs, n_batch, bat_per_epo, n_steps))
    
    # manually enumerate epochs
    f_history = open(f"{LOG_PATH}SSL_GAN.csv", "w")
    f_history.write("step,generator_loss,unsupervised_real_loss,unsupervised_fake_loss,supervised_loss,supervised_acc,train_loss,test_loss,train_acc,test_acc\n")
    
    #for epoch in n_epochs:
    #    for batch in range(bat_per_epo):
    for step in range(1,n_steps+1):
#         t_start = time.time()
        # update supervised discriminator (c)
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], n_batch)
        c_loss, c_acc = supervised_model.train_on_batch(Xsup_real, ysup_real)
        
        # update unsupervised discriminator (d)
        [X_real, _], y_real = generate_real_samples(dataset_train, n_batch)
        d_loss1, real_acc = unsupervised_model.train_on_batch(X_real, y_real)
        
        X_fake, y_fake = generate_fake_samples(generator_model, latent_dim, n_batch)
        d_loss2, fake_acc = unsupervised_model.train_on_batch(X_fake, y_fake)
        
        # update generator (g)
        X_gan, y_gan = generate_latent_points(latent_dim, n_batch), np.ones((n_batch, 1))
        g_loss = gan_model.train_on_batch(X_gan, y_gan)
#         t_total = (time.time() - t_start)
        # summarize loss on this batch
    
        # Train - Test
        X_train, y_train = dataset_train
        loss_train, acc_train = supervised_model.evaluate(X_train, y_train, verbose=0)

        # evaluate the test classifier model
        X_test, y_test = dataset_test
        loss_test, acc_test = supervised_model.evaluate(X_test, y_test, verbose=0)
        
        # Log
        print('epoch: %d | step: %d | Train: G_Loss: %.3f, ' \
              'D_unsup_loss_real: %.3f, D_unsup_acc_real:  %.2f, ' \
              'D_unsup_loss_fake: %.3f, D_unsup_acc_fake: %.2f, ' \
              'D_sup_loss: %.3f, D_sup_acc: %.2f ' \
              'Train acc: %.3f Test acc: %.3f ' %(int(step/bat_per_epo), step, g_loss,
                                                d_loss1, real_acc*100,
                                                d_loss2, fake_acc*100,
                                                c_loss, c_acc*100,
                                                 acc_train*100, acc_test*100))#, end = '\r')
        f_history.write(f"{step},{g_loss},{d_loss1},{d_loss2},{c_loss},{c_acc*100},{loss_train},{loss_test},{acc_train*100},{acc_test*100}\n")
        
        if step==1:
            plot_data(unnormalize(X_test).astype(int), 0, "test", grid_size = [10, 10], OUT_PATH=LOG_PATH)
        # evaluate the model performance every so often
        if (step) % (100) == 0 or step == 1:
            #summarize_performance(step, generator_model, supervised_model, latent_dim, dataset, dataset_test)
            # prepare fake examples
            X_generated, _ = generate_fake_samples(generator_model, latent_dim, n_samples=100)
            # scale from [-1,1] to [0,255]
            plot_data(unnormalize(X_generated).astype(int), step, "generated", grid_size = [10, 10], OUT_PATH=LOG_PATH)
            
            X_train, y_train = dataset_train
            _, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
            print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))
            
            # evaluate the test classifier model
            X_test, y_test = dataset_test
            _, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
            print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))
            # save the generator model
            filename2 = f'{LOG_PATH}generator_model_{step}.h5'
            generator_model.save(filename2)
            # save the classifier model
            filename3 = f'{LOG_PATH}supervised_model_{step}.h5'
            supervised_model.save(filename3)
            print('>Saving models Generator: %s and Supervised: %s' % (filename2, filename3))
    
    f_history.close()

### Parameters

In [12]:
input_shape = (32, 32, 3)
num_classes = 10

learning_rate =  0.0002
latent_dim = 100

labeled_rate = 1.0
labeled_samples = 50000*labeled_rate

In [13]:
LOG_PATH = f"Logs/SSGAN_CIFAR10/Classifier_{int(labeled_samples)}/"
verifyDir(LOG_PATH)

### Creating models

In [14]:
# create the discriminator models
unsupervised_model, supervised_model = define_discriminator(in_shape=input_shape, n_classes=num_classes, learning_rate = learning_rate)
# create the generator
generator_model = define_generator(latent_dim=latent_dim)

In [15]:
supervised_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496 

In [16]:
unsupervised_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        1849

In [17]:
generator_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 256)         1048832   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       5244

In [18]:
# create the gan
gan_model = define_gan(generator_model, unsupervised_model, learning_rate = learning_rate)

In [19]:
gan_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              413696    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 4096)              0         
_________________________________________________________________
reshape (Reshape)            (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 8, 8, 256)         1048832   
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 16, 16, 128)       5244

### Loading Dataset

In [20]:
# load image data
dataset_train, dataset_test = load_real_samples()

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)


### Training

In [21]:
epochs=300
batch_size=128

In [ ]:
train(generator_model, unsupervised_model, supervised_model, gan_model, 
      dataset_train, dataset_test, latent_dim=latent_dim, 
      n_epochs=epochs, n_batch=batch_size, percent_samples=labeled_rate, n_classes=num_classes)

Sup samples: (50000, 32, 32, 3) (50000,)
n_epochs=300, n_batch=128, batch/epoch=390, steps=117000
epoch: 0 | step: 1 | Train: G_Loss: 0.095, D_unsup_loss_real: 0.094, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.402, D_unsup_acc_fake: 0.00, D_sup_loss: 2.304, D_sup_acc: 9.38 Train acc: 10.464 Test acc: 10.500 
Train Classifier Accuracy: 10.464%

Test Classifier Accuracy: 10.500%

>Saving models Generator: Logs/SSGAN_CIFAR10/Classifier_50000/generator_model_1.h5 and Supervised: Logs/SSGAN_CIFAR10/Classifier_50000/supervised_model_1.h5
epoch: 0 | step: 2 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.092, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.398, D_unsup_acc_fake: 0.00, D_sup_loss: 2.302, D_sup_acc: 10.50 Train acc: 10.352 Test acc: 10.290 
epoch: 0 | step: 3 | Train: G_Loss: 0.096, D_unsup_loss_real: 0.089, D_unsup_acc_real:  100.00, D_unsup_loss_fake: 2.396, D_unsup_acc_fake: 0.00, D_sup_loss: 2.300, D_sup_acc: 10.30 Train acc: 13.208 Test acc: 13.100 
epoch: 0 | step: 4 | 

epoch: 0 | step: 37 | Train: G_Loss: 3.223, D_unsup_loss_real: 0.286, D_unsup_acc_real:  85.94, D_unsup_loss_fake: 0.179, D_unsup_acc_fake: 100.00, D_sup_loss: 2.305, D_sup_acc: 17.32 Train acc: 18.338 Test acc: 18.670 
epoch: 0 | step: 38 | Train: G_Loss: 3.442, D_unsup_loss_real: 0.279, D_unsup_acc_real:  84.38, D_unsup_loss_fake: 0.161, D_unsup_acc_fake: 100.00, D_sup_loss: 2.266, D_sup_acc: 18.59 Train acc: 18.730 Test acc: 19.240 
epoch: 0 | step: 39 | Train: G_Loss: 3.297, D_unsup_loss_real: 0.344, D_unsup_acc_real:  82.03, D_unsup_loss_fake: 0.187, D_unsup_acc_fake: 100.00, D_sup_loss: 2.280, D_sup_acc: 19.19 Train acc: 16.740 Test acc: 16.530 
epoch: 0 | step: 40 | Train: G_Loss: 3.500, D_unsup_loss_real: 0.238, D_unsup_acc_real:  83.59, D_unsup_loss_fake: 0.099, D_unsup_acc_fake: 100.00, D_sup_loss: 2.349, D_sup_acc: 16.48 Train acc: 15.442 Test acc: 15.560 
epoch: 0 | step: 41 | Train: G_Loss: 3.309, D_unsup_loss_real: 0.343, D_unsup_acc_real:  82.81, D_unsup_loss_fake: 0.264

epoch: 0 | step: 75 | Train: G_Loss: 1.415, D_unsup_loss_real: 0.503, D_unsup_acc_real:  74.22, D_unsup_loss_fake: 0.405, D_unsup_acc_fake: 96.09, D_sup_loss: 2.742, D_sup_acc: 23.16 Train acc: 20.662 Test acc: 20.560 
epoch: 0 | step: 76 | Train: G_Loss: 1.453, D_unsup_loss_real: 0.408, D_unsup_acc_real:  76.56, D_unsup_loss_fake: 0.367, D_unsup_acc_fake: 99.22, D_sup_loss: 2.546, D_sup_acc: 20.55 Train acc: 19.904 Test acc: 19.950 
epoch: 0 | step: 77 | Train: G_Loss: 1.678, D_unsup_loss_real: 0.296, D_unsup_acc_real:  81.25, D_unsup_loss_fake: 0.312, D_unsup_acc_fake: 100.00, D_sup_loss: 2.562, D_sup_acc: 19.84 Train acc: 18.054 Test acc: 18.300 
epoch: 0 | step: 78 | Train: G_Loss: 1.672, D_unsup_loss_real: 0.465, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.303, D_unsup_acc_fake: 100.00, D_sup_loss: 2.592, D_sup_acc: 18.25 Train acc: 20.020 Test acc: 20.320 
epoch: 0 | step: 79 | Train: G_Loss: 1.688, D_unsup_loss_real: 0.240, D_unsup_acc_real:  89.84, D_unsup_loss_fake: 0.266, 

epoch: 0 | step: 112 | Train: G_Loss: 1.907, D_unsup_loss_real: 0.318, D_unsup_acc_real:  81.25, D_unsup_loss_fake: 0.273, D_unsup_acc_fake: 99.22, D_sup_loss: 2.601, D_sup_acc: 18.47 Train acc: 17.798 Test acc: 18.540 
epoch: 0 | step: 113 | Train: G_Loss: 1.880, D_unsup_loss_real: 0.379, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 0.302, D_unsup_acc_fake: 99.22, D_sup_loss: 2.573, D_sup_acc: 18.49 Train acc: 20.504 Test acc: 21.640 
epoch: 0 | step: 114 | Train: G_Loss: 1.822, D_unsup_loss_real: 0.364, D_unsup_acc_real:  80.47, D_unsup_loss_fake: 0.321, D_unsup_acc_fake: 96.88, D_sup_loss: 2.432, D_sup_acc: 21.60 Train acc: 17.184 Test acc: 18.270 
epoch: 0 | step: 115 | Train: G_Loss: 1.845, D_unsup_loss_real: 0.360, D_unsup_acc_real:  83.59, D_unsup_loss_fake: 0.342, D_unsup_acc_fake: 95.31, D_sup_loss: 2.389, D_sup_acc: 18.22 Train acc: 18.942 Test acc: 19.670 
epoch: 0 | step: 116 | Train: G_Loss: 1.626, D_unsup_loss_real: 0.406, D_unsup_acc_real:  78.91, D_unsup_loss_fake: 0.37

epoch: 0 | step: 150 | Train: G_Loss: 2.806, D_unsup_loss_real: 0.420, D_unsup_acc_real:  80.47, D_unsup_loss_fake: 0.191, D_unsup_acc_fake: 99.22, D_sup_loss: 2.259, D_sup_acc: 21.53 Train acc: 20.852 Test acc: 21.100 
epoch: 0 | step: 151 | Train: G_Loss: 2.819, D_unsup_loss_real: 0.274, D_unsup_acc_real:  87.50, D_unsup_loss_fake: 0.253, D_unsup_acc_fake: 92.19, D_sup_loss: 2.256, D_sup_acc: 21.06 Train acc: 19.608 Test acc: 20.110 
epoch: 0 | step: 152 | Train: G_Loss: 3.879, D_unsup_loss_real: 0.244, D_unsup_acc_real:  90.62, D_unsup_loss_fake: 0.976, D_unsup_acc_fake: 59.38, D_sup_loss: 2.331, D_sup_acc: 20.05 Train acc: 16.688 Test acc: 17.380 
epoch: 0 | step: 153 | Train: G_Loss: 3.257, D_unsup_loss_real: 0.588, D_unsup_acc_real:  70.31, D_unsup_loss_fake: 0.896, D_unsup_acc_fake: 53.12, D_sup_loss: 2.376, D_sup_acc: 17.37 Train acc: 18.804 Test acc: 18.860 
epoch: 0 | step: 154 | Train: G_Loss: 2.955, D_unsup_loss_real: 0.571, D_unsup_acc_real:  75.00, D_unsup_loss_fake: 1.10

...

epoch: 2 | step: 1091 | Train: G_Loss: 1.134, D_unsup_loss_real: 0.491, D_unsup_acc_real:  71.88, D_unsup_loss_fake: 0.652, D_unsup_acc_fake: 63.28, D_sup_loss: 1.442, D_sup_acc: 47.62 Train acc: 47.708 Test acc: 47.380 
epoch: 2 | step: 1092 | Train: G_Loss: 1.204, D_unsup_loss_real: 0.528, D_unsup_acc_real:  68.75, D_unsup_loss_fake: 0.596, D_unsup_acc_fake: 73.44, D_sup_loss: 1.440, D_sup_acc: 47.33 Train acc: 47.780 Test acc: 47.480 
epoch: 2 | step: 1093 | Train: G_Loss: 1.132, D_unsup_loss_real: 0.585, D_unsup_acc_real:  60.16, D_unsup_loss_fake: 0.609, D_unsup_acc_fake: 71.09, D_sup_loss: 1.447, D_sup_acc: 47.42 Train acc: 47.940 Test acc: 47.440 
epoch: 2 | step: 1094 | Train: G_Loss: 1.144, D_unsup_loss_real: 0.595, D_unsup_acc_real:  63.28, D_unsup_loss_fake: 0.702, D_unsup_acc_fake: 54.69, D_sup_loss: 1.441, D_sup_acc: 47.43 Train acc: 47.588 Test acc: 47.230 
epoch: 2 | step: 1095 | Train: G_Loss: 1.122, D_unsup_loss_real: 0.483, D_unsup_acc_real:  68.75, D_unsup_loss_fake:

epoch: 2 | step: 1128 | Train: G_Loss: 1.199, D_unsup_loss_real: 0.650, D_unsup_acc_real:  58.59, D_unsup_loss_fake: 0.637, D_unsup_acc_fake: 67.19, D_sup_loss: 1.446, D_sup_acc: 47.36 Train acc: 47.390 Test acc: 46.820 
epoch: 2 | step: 1129 | Train: G_Loss: 1.162, D_unsup_loss_real: 0.526, D_unsup_acc_real:  64.84, D_unsup_loss_fake: 0.726, D_unsup_acc_fake: 65.62, D_sup_loss: 1.456, D_sup_acc: 46.76 Train acc: 47.814 Test acc: 47.510 
epoch: 2 | step: 1130 | Train: G_Loss: 1.146, D_unsup_loss_real: 0.501, D_unsup_acc_real:  71.09, D_unsup_loss_fake: 0.561, D_unsup_acc_fake: 75.78, D_sup_loss: 1.447, D_sup_acc: 47.51 Train acc: 47.590 Test acc: 47.230 
epoch: 2 | step: 1131 | Train: G_Loss: 1.177, D_unsup_loss_real: 0.596, D_unsup_acc_real:  57.81, D_unsup_loss_fake: 0.617, D_unsup_acc_fake: 69.53, D_sup_loss: 1.454, D_sup_acc: 47.16 Train acc: 47.148 Test acc: 46.550 
epoch: 2 | step: 1132 | Train: G_Loss: 1.119, D_unsup_loss_real: 0.531, D_unsup_acc_real:  65.62, D_unsup_loss_fake:

epoch: 2 | step: 1166 | Train: G_Loss: 1.225, D_unsup_loss_real: 0.471, D_unsup_acc_real:  73.44, D_unsup_loss_fake: 0.715, D_unsup_acc_fake: 60.94, D_sup_loss: 1.441, D_sup_acc: 47.73 Train acc: 48.498 Test acc: 48.170 
epoch: 2 | step: 1167 | Train: G_Loss: 1.246, D_unsup_loss_real: 0.484, D_unsup_acc_real:  73.44, D_unsup_loss_fake: 0.623, D_unsup_acc_fake: 74.22, D_sup_loss: 1.433, D_sup_acc: 48.20 Train acc: 47.828 Test acc: 47.570 
epoch: 2 | step: 1168 | Train: G_Loss: 1.229, D_unsup_loss_real: 0.524, D_unsup_acc_real:  66.41, D_unsup_loss_fake: 0.594, D_unsup_acc_fake: 71.88, D_sup_loss: 1.444, D_sup_acc: 47.49 Train acc: 47.992 Test acc: 47.670 
epoch: 2 | step: 1169 | Train: G_Loss: 1.328, D_unsup_loss_real: 0.562, D_unsup_acc_real:  66.41, D_unsup_loss_fake: 0.683, D_unsup_acc_fake: 67.19, D_sup_loss: 1.450, D_sup_acc: 47.59 Train acc: 47.804 Test acc: 47.500 
epoch: 3 | step: 1170 | Train: G_Loss: 1.351, D_unsup_loss_real: 0.512, D_unsup_acc_real:  69.53, D_unsup_loss_fake:

epoch: 3 | step: 1203 | Train: G_Loss: 1.523, D_unsup_loss_real: 0.476, D_unsup_acc_real:  67.19, D_unsup_loss_fake: 0.857, D_unsup_acc_fake: 54.69, D_sup_loss: 1.445, D_sup_acc: 47.51 Train acc: 48.242 Test acc: 47.320 
epoch: 3 | step: 1204 | Train: G_Loss: 1.531, D_unsup_loss_real: 0.613, D_unsup_acc_real:  58.59, D_unsup_loss_fake: 0.607, D_unsup_acc_fake: 71.09, D_sup_loss: 1.452, D_sup_acc: 47.27 Train acc: 48.384 Test acc: 47.620 
epoch: 3 | step: 1205 | Train: G_Loss: 1.457, D_unsup_loss_real: 0.692, D_unsup_acc_real:  51.56, D_unsup_loss_fake: 0.601, D_unsup_acc_fake: 68.75, D_sup_loss: 1.444, D_sup_acc: 47.54 Train acc: 48.608 Test acc: 47.730 
epoch: 3 | step: 1206 | Train: G_Loss: 1.318, D_unsup_loss_real: 0.545, D_unsup_acc_real:  65.62, D_unsup_loss_fake: 0.628, D_unsup_acc_fake: 68.75, D_sup_loss: 1.436, D_sup_acc: 47.66 Train acc: 47.516 Test acc: 46.980 
epoch: 3 | step: 1207 | Train: G_Loss: 1.320, D_unsup_loss_real: 0.673, D_unsup_acc_real:  55.47, D_unsup_loss_fake:

epoch: 3 | step: 1241 | Train: G_Loss: 1.284, D_unsup_loss_real: 0.616, D_unsup_acc_real:  62.50, D_unsup_loss_fake: 0.623, D_unsup_acc_fake: 71.09, D_sup_loss: 1.425, D_sup_acc: 48.90 Train acc: 49.430 Test acc: 48.840 
epoch: 3 | step: 1242 | Train: G_Loss: 1.453, D_unsup_loss_real: 0.543, D_unsup_acc_real:  66.41, D_unsup_loss_fake: 0.644, D_unsup_acc_fake: 64.06, D_sup_loss: 1.416, D_sup_acc: 48.85 Train acc: 49.550 Test acc: 48.770 
epoch: 3 | step: 1243 | Train: G_Loss: 1.295, D_unsup_loss_real: 0.601, D_unsup_acc_real:  57.81, D_unsup_loss_fake: 0.556, D_unsup_acc_fake: 78.91, D_sup_loss: 1.413, D_sup_acc: 48.72 Train acc: 49.242 Test acc: 48.520 


### Testing

In [ ]:
dataset_train, dataset_test = load_real_samples()

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
last_step = int(dataset_train.shape[0]/batch_size)*epochs

In [ ]:
supervised_model = load_model(f'{LOG_PATH}supervised_model_{last_step}.h5')

In [ ]:
X_train, y_train = dataset_train
_, acc = supervised_model.evaluate(X_train, y_train, verbose=0)
print('Train Classifier Accuracy: %.3f%%\n' % (acc * 100))

In [ ]:
X_test, y_test = dataset_test
_, acc = supervised_model.evaluate(X_test, y_test, verbose=0)
print('Test Classifier Accuracy: %.3f%%\n' % (acc * 100))

### Plotting

In [ ]:
import pandas as pd

In [ ]:
results_file = pd.read_csv(f"{LOG_PATH}SSL_GAN.csv")

In [ ]:
log_file = results_file.iloc[:,1:]
log_file

In [ ]:
log_file.iloc[:, [0,1,2,3]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [5,6]].plot(figsize=(12,8))

In [ ]:
log_file.iloc[:, [7,8]].plot(figsize=(12,8), ylim=(0,100), yticks=range(0,110,10))